# Detailed Report on Flood Detection Model for Disaster Management

## Problem Statement

The objective of this project was to develop a computer vision model that can effectively segment images to identify areas affected by flooding. This task is crucial for aiding disaster management efforts by providing accurate information about the extent and location of flood-impacted regions. By distinguishing between flooded and non-flooded areas within an image, the model supports critical activities such as planning rescue operations, assessing damage, and prioritizing resources during flood events. Furthermore, the segmented data can be utilized for post-event analysis and future flood prediction and planning.

## Approach and Methodology

### Data Preparation

The dataset provided consists of training and testing images along with their corresponding segmentation masks. These masks define the flooded areas within each image. Initial steps involved preprocessing the data to normalize the image sizes and pixel values, ensuring consistency across the dataset. Data augmentation techniques such as rotations, scaling, and horizontal flipping were employed to enhance model robustness and prevent overfitting.

### Model Architecture: U-Net

The U-Net architecture was chosen due to its proven efficacy in tasks requiring precise localization, such as medical image segmentation. This architecture is particularly suitable for segmenting small objects and detailed textures in images, which is analogous to identifying nuanced differences in flooded areas.

#### Key Features of U-Net:
- **Symmetric Structure:** The architecture consists of a contracting path to capture context and a symmetric expanding path that enables precise localization.
- **Skip Connections:** These connections between layers of equal resolution in the contracting and expanding paths help the model retain important high-resolution features.

### Training Process

#### Loss Function:
- **BCEWithLogitsLoss:** This loss function combines a sigmoid layer and the binary cross-entropy loss in one single class, which is particularly suitable for binary classification tasks like segmentation.

#### Optimizer:
- **AdamW Optimizer:** Known for its effectiveness in deep learning tasks, AdamW was chosen due to its ability to combine the benefits of Adam optimization with weight decay regulation, providing better control over learning.

### Parameters:
- **Epochs:** The model was trained over 10 epochs.
- **Learning Rate:** Initially set at 0.001.
- **Batch Size:** Determined by the computational limits of the training environment, aiming for a balance between speed and memory usage.

## Results and Evaluation

After training for 10 epochs, the model achieved an **Average IoU of 0.68** on the test set. This metric, which ranges from 0 to 1, measures the overlap between the predicted segmentation and the actual mask, with 1 representing perfect overlap and 0 representing no overlap.

## Conclusion

The U-Net model demonstrated a promising ability to segment flooded areas from aerial images, achieving a moderate IoU score. This performance indicates that the model can effectively contribute to disaster management efforts by providing reliable data for assessing and responding to flood situations.


# Imports

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import os
import pandas as pd
import torchvision.transforms as transforms
import torch.optim as optim
from torch.optim import AdamW
from torch.optim.lr_scheduler import StepLR

# Model Arch

In [ ]:
class UNet(nn.Module):
    def __init__(self, in_channels=3, out_channels=1):
        super(UNet, self).__init__()

        def CBR(in_ch, out_ch, kernel_size=3, stride=1, padding=1, dropout_rate=0.1):
            return nn.Sequential(
                nn.Conv2d(in_ch, out_ch, kernel_size, stride, padding),
                nn.BatchNorm2d(out_ch),
                nn.ReLU(inplace=True),
                nn.Dropout(dropout_rate)
            )

        def encoder(in_ch, out_ch):
            return nn.Sequential(
                CBR(in_ch, out_ch),
                CBR(out_ch, out_ch),
                nn.MaxPool2d(2)
            )

        def decoder(in_ch, out_ch, kernel_size=2, stride=2, padding=0):
            return nn.Sequential(
                nn.ConvTranspose2d(in_ch, out_ch, kernel_size, stride, padding),
                CBR(out_ch, out_ch),
                CBR(out_ch, out_ch),
            )

        self.enc1 = encoder(in_channels, 64)
        self.enc2 = encoder(64, 128)
        self.enc3 = encoder(128, 256)
        self.enc4 = encoder(256, 512)

        self.bottleneck = nn.Sequential(
            CBR(512, 1024),
            CBR(1024, 1024),
            CBR(1024, 1024),  # Added an extra CBR layer to match the output channels
        )

        self.dec1 = decoder(1536, 1024)  # bottleneck (1024) + enc4 (512) => 1536
        self.dec2 = decoder(1024 + 256, 512)
        self.dec3 = decoder(512 + 128, 256)
        self.dec4 = decoder(256 + 64, 128)

        self.final = nn.Conv2d(128, out_channels, kernel_size=1)

    def forward(self, x):
        enc1 = self.enc1(x)
        enc2 = self.enc2(enc1)
        enc3 = self.enc3(enc2)
        enc4 = self.enc4(enc3)

        bottleneck = self.bottleneck(enc4)

        def crop_and_concat(up_tensor, down_tensor):
            _, _, H, W = up_tensor.shape
            down_tensor_cropped = torchvision.transforms.CenterCrop([H, W])(down_tensor)
            return torch.cat([up_tensor, down_tensor_cropped], 1)

        dec1 = self.dec1(crop_and_concat(bottleneck, enc4))
        dec2 = self.dec2(crop_and_concat(dec1, enc3))
        dec3 = self.dec3(crop_and_concat(dec2, enc2))
        dec4 = self.dec4(crop_and_concat(dec3, enc1))

        return self.final(dec4)

# Dataset Preparation

In [ ]:
class FloodDataset(Dataset):
    def __init__(self, csv_file, img_dir, mask_dir, image_transform=None, mask_transform=None):
        self.img_labels = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.image_transform = image_transform
        self.mask_transform = mask_transform

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_name = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        mask_name = os.path.join(self.mask_dir, self.img_labels.iloc[idx, 1])
        image = Image.open(img_name).convert("RGB")
        mask = Image.open(mask_name).convert("L")

        if self.image_transform:
            image = self.image_transform(image)

        if self.mask_transform:
            mask = self.mask_transform(mask)

        return image, mask

In [ ]:
data_augmentation = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
])

image_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

mask_transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

train_dataset = FloodDataset(
    csv_file='/content/drive/MyDrive/DLCV Project/Flood-Detection-Project/train-metadata.csv',
    img_dir='/content/drive/MyDrive/DLCV Project/Flood-Detection-Project/Image/Train-images',
    mask_dir='/content/drive/MyDrive/DLCV Project/Flood-Detection-Project/Mask/Train-Masks',
    image_transform=image_transform,
    mask_transform=mask_transform
)

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

test_dataset = FloodDataset(
    csv_file='/content/drive/MyDrive/DLCV Project/Flood-Detection-Project/test-metadata.csv',
    img_dir='/content/drive/MyDrive/DLCV Project/Flood-Detection-Project/Image/Test-images',
    mask_dir='/content/drive/MyDrive/DLCV Project/Flood-Detection-Project/Mask/Test-Masks',
    image_transform=image_transform,
    mask_transform=mask_transform
)

test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False)

# Training

In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        BCE_loss = nn.BCEWithLogitsLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss
        return F_loss.mean()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet(in_channels=3, out_channels=1).to(device)
criterion = FocalLoss()

optimizer = AdamW(model.parameters(), lr=1e-3, weight_decay=1e-3)
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    for images, masks in train_loader:
        images, masks = images.to(device), masks.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

Epoch 1, Loss: 0.022475264966487885
Epoch 2, Loss: 0.027996350079774857
Epoch 3, Loss: 0.01992657780647278
Epoch 4, Loss: 0.03443533927202225
Epoch 5, Loss: 0.021336844190955162
Epoch 6, Loss: 0.018526969477534294
Epoch 7, Loss: 0.04886858910322189
Epoch 8, Loss: 0.024182312190532684
Epoch 9, Loss: 0.017775293439626694
Epoch 10, Loss: 0.03204552084207535


# Test IoU

In [ ]:
def calculate_iou(preds, labels):
    smooth = 1e-6
    preds = torch.sigmoid(preds)
    preds = (preds > 0.5).float()
    preds = preds.int()
    labels = labels.int()

    intersection = (preds & labels).sum((1, 2))
    union = (preds | labels).sum((1, 2))

    iou = (intersection + smooth) / (union + smooth)
    return iou.mean()

model.eval()
total_iou = 0
count = 0

with torch.no_grad():
    for images, masks in test_loader:
        images = images.to(device)
        masks = masks.to(device)
        outputs = model(images)

        iou = calculate_iou(outputs, masks)
        total_iou += iou
        count += 1

average_iou = total_iou / count
print(f'Average IoU on the test set: {average_iou.item()}')

Average IoU on the test set: 0.6501545310020447
